# Test GPU

In [ ]:
import torch
torch.cuda.current_device()
torch.cuda.get_device_name(0)

# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importing Necessary Libraries

In [3]:
import shutil
import os
import h5py
import csv

# importing the libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import time
import glob
import gc

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *
import torch.utils.data

%matplotlib inline


# Copy Data

In [ ]:
shutil.copyfile("/content/drive/My Drive/Alzheimer Competition/test_cloud.zip" , "test_cloud.zip")
shutil.copyfile("/content/drive/My Drive/Alzheimer Competition/submission_format.csv", "submission_format.csv")

!unzip test_cloud.zip;

os.remove("test_cloud.zip")

path = "./test_cloud/"
submission_format_csv = "submission_format.csv"

# Model

### CNN Model

In [ ]:
depth, height, width = 32, 64, 64   # dimension for converting point cloud to voxels

shutil.copyfile("/content/drive/My Drive/Alzheimer Competition/cloud_size_32_64_64_acc_81.198_mcc_0.515.pth", "weight_3D.pth")
checkpoint_model = "weight_3D.pth"

In [ ]:
num_classes = 2

# Create CNN Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.conv_layer1 = self._conv_layer_set(3, 32)
        self.conv_layer2 = self._conv_layer_set(32, 64)
        self.fc1 = nn.Linear(75264, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, num_classes)
        self.relu = nn.LeakyReLU()
        self.batch1 = nn.BatchNorm1d(2048)
        self.batch2 = nn.BatchNorm1d(512)
        self.batch3 = nn.BatchNorm1d(128)
        self.drop=nn.Dropout(p=0.5)        
        
    def _conv_layer_set(self, in_c, out_c):
        conv_layer = nn.Sequential(
        nn.Conv3d(in_c, out_c, kernel_size=(3, 3, 3), padding=0),
        nn.LeakyReLU(),
        nn.MaxPool3d((2, 2, 2)),
        )
        return conv_layer
    

    def forward(self, x):
        # Set 1
        out = self.conv_layer1(x)
        out = self.drop(out)
        out = self.conv_layer2(out)
        out = out.view(out.size(0), -1)
        #print(out.shape)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.batch1(out)
        out = self.drop(out)
        
        out = self.fc2(out)
        out = self.relu(out)
        out = self.batch2(out)
        out = self.drop(out)

        out = self.fc3(out)
        out = self.relu(out)
        out = self.batch3(out)
        out = self.drop(out)
        out = self.fc4(out)
        
        return out


# Create CNN
model = CNNModel()
#model.cuda()
print(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
#device


#loading saved weight file
model.load_state_dict(torch.load(checkpoint_model))


#device

### Resnet3D  18 Model


In [ ]:
depth, height, width = 32, 64, 64   # dimension for converting point cloud to voxels

shutil.copyfile("/content/drive/My Drive/Alzheimer Competition/cloud_size_32_64_64_acc_81.198_mcc_0.515.pth", "weight_3D.pth")
checkpoint_model = "weight_3D.pth"

In [ ]:
from torchvision.models.video import r3d_18
#from torchvision.models.video import r2plus1d_18
model = r3d_18(pretrained = False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.fc.out_features = 2

model.load_state_dict(torch.load(checkpoint_model))

# Dataset

### Functions

In [7]:
def pc2voxel(cloud0, cloud1, cloud2, depth=16, height=32, width=32):

    voxel_grid = np.zeros((3, depth, height, width), dtype=np.float16)

    in_depth = max(np.max(cloud0[:, 0]), np.max(cloud1[:, 0]), np.max(cloud2[:, 0]))
    in_height = max(np.max(cloud0[:, 1]), np.max(cloud1[:, 1]), np.max(cloud2[:, 1]))
    in_width = max(np.max(cloud0[:, 2]), np.max(cloud1[:, 2]), np.max(cloud2[:, 2]))

    if in_depth >= depth:
        depth_ratio = depth / (in_depth + 1)
        cloud0[:, 0] = np.uint32(cloud0[:, 0].astype(float) * depth_ratio)
        cloud1[:, 0] = np.uint32(cloud1[:, 0].astype(float) * depth_ratio)
        cloud2[:, 0] = np.uint32(cloud2[:, 0].astype(float) * depth_ratio)
    if in_height >= height:
        height_ratio = height / (in_height + 1)
        cloud0[:, 1] = np.uint32(cloud0[:, 1].astype(float) * height_ratio)
        cloud1[:, 1] = np.uint32(cloud1[:, 1].astype(float) * height_ratio)
        cloud2[:, 1] = np.uint32(cloud2[:, 1].astype(float) * height_ratio)
    if in_width >= width:
        width_ratio = width / (in_width + 1)
        cloud0[:, 2] = np.uint32(cloud0[:, 2].astype(float) * width_ratio)
        cloud1[:, 2] = np.uint32(cloud1[:, 2].astype(float) * width_ratio)
        cloud2[:, 2] = np.uint32(cloud2[:, 2].astype(float) * width_ratio)

    voxel_grid[0, cloud0[:, 0], cloud0[:, 1], cloud0[:, 2]] = 1.0
    voxel_grid[1, cloud1[:, 0], cloud1[:, 1], cloud1[:, 2]] = 1.0
    voxel_grid[2, cloud2[:, 0], cloud2[:, 1], cloud2[:, 2]] = 1.0

    return voxel_grid

In [8]:
def string2torch(test_y):
  df = pd.DataFrame(test_y, columns = ['Fname'])
  y = (df['Fname'].values)

  for i,filename in enumerate(y):
    seq_name = filename.split(".mp4")[0]
    y[i] = seq_name

  processed = np.array(y)
  processed = processed.astype(np.int)
  processed = torch.from_numpy(processed)

  return processed

In [9]:
def make_submission_file(filenames, y_pred):
  submit = []
  filenames = filenames.astype(int)
  for i in filenames:
    submit += [str(i)+'.mp4']

  submission_dict = {"filename": submit, "stalled": y_pred.astype(int)}
  submission_csv = pd.DataFrame(submission_dict)
  submission_csv.to_csv("submission3D.csv", index=False)
  

### Loading

In [ ]:
with open(submission_format_csv, mode='r') as infile:
    reader = csv.reader(infile)
    test_list_csv = {rows[0]: rows[1] for rows in reader}
    infile.close()


files = list(test_list_csv.keys())
files.pop(0)
test_len = len(files)
print("Total data: " + str(test_len))

# Prediction and Submission file generation

### CNN Submission

In [ ]:
y_pred = []
filenames = np.array([])


batch_size = 128

big_batch_size = 1000   # number of test data loaded at a time

for big_batch_no in range(math.ceil(test_len/big_batch_size)):

  this_batch_len = big_batch_size
  if (test_len - big_batch_size * big_batch_no) < big_batch_size:
    this_batch_len = test_len - big_batch_size * big_batch_no


  test_x = np.zeros((this_batch_len, 3, depth, height, width), dtype=np.float32)
  test_yy = np.zeros(this_batch_len)


  # Load one big batch
  for i in range(this_batch_len):

    original_idx = i + big_batch_size * big_batch_no

    f = files[original_idx]
    original_name = f.replace(".mp4", "")
    h5_filename = path + original_name + ".h5"

    hf = h5py.File(h5_filename, 'r')
    c1 = hf['cloud1'][:]
    c2 = hf['cloud2'][:]
    c3 = hf['cloud3'][:]
    hf.close()


    test_x[i, :, :, :, :] = pc2voxel(c1, c2, c3, depth=depth, height=height, width=width)
    test_yy[i] = int(original_name)
    # print(original_idx, original_name, f, h5_filename)

  
  # Test one big batch
  test_y = torch.from_numpy(test_yy).int()
  ## change the data types from here
  test_x = torch.from_numpy(test_x).float()
  #test_y = torch.from_numpy(test_y)
  
  test = torch.utils.data.TensorDataset(test_x, test_y)
  test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False, num_workers=4)

  model.eval()
  for i,(images,labels) in tqdm(enumerate(test_loader)):
      print(labels)
      filenames = np.append(filenames, labels)
      
      images = images.view(-1,3,depth,height,width)
      test = Variable(images.to(device), requires_grad=False)
      labels = Variable(labels.to(device), requires_grad=False)

      with torch.no_grad():
        # Forward propagation
        outputs = model(test)

        # Get predictions from the maximum value
        predicted = torch.max(outputs.data, 1)[1]
        #print(f"prediction size are {predicted.shape}")
        y_pred = np.append(y_pred, predicted.cpu().numpy())

#print(filenames)

make_submission_file(filenames, y_pred)
'''
submission_dict = {"filename": filenames, "stalled": y_pred.astype(int)}

submission_csv = pd.DataFrame(submission_dict)

submission_csv.to_csv("submission_file.csv", index=False)
''' 

### Resnet3D 18 Submission

In [ ]:
y_pred = []
filenames = np.array([])


batch_size = 100

big_batch_size = 10   # number of test data loaded at a time

for big_batch_no in range(math.ceil(test_len/big_batch_size)):

  this_batch_len = big_batch_size
  if (test_len - big_batch_size * big_batch_no) < big_batch_size:
    this_batch_len = test_len - big_batch_size * big_batch_no


  test_x = np.zeros((this_batch_len, 3, depth, height, width), dtype=np.float32)
  test_yy = np.zeros(this_batch_len)


  # Load one big batch
  for i in tqdm(range(this_batch_len)):

    original_idx = i + big_batch_size * big_batch_no

    f = files[original_idx]
    original_name = f.replace(".mp4", "")
    h5_filename = path + original_name + ".h5"

    hf = h5py.File(h5_filename, 'r')
    c1 = hf['cloud1'][:]
    c2 = hf['cloud2'][:]
    c3 = hf['cloud3'][:]
    hf.close()


    test_x[i, :, :, :, :] = pc2voxel(c1, c2, c3, depth=depth, height=height, width=width)
    test_yy[i] = int(original_name)

    # print(original_idx, original_name, f, h5_filename)

  
  # Test one big batch
  test_y = torch.from_numpy(test_yy).int()
  ## change the data types from here
  test_x = torch.from_numpy(test_x).float()
  #test_y = torch.from_numpy(test_y)
  
  test = torch.utils.data.TensorDataset(test_x, test_y)
  test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False, num_workers=4)

  model.eval()
  for i,(images,labels) in tqdm(enumerate(test_loader)):
      #print(labels)
      filenames = np.append(filenames, labels)
      
      images = images.view(-1,3,depth,height,width)
      test = Variable(images.to(device), requires_grad=False)
      labels = Variable(labels.to(device), requires_grad=False)

      with torch.no_grad():
        # Forward propagation
        outputs = model(test)

        # Get predictions from the maximum value
        predicted = torch.max(outputs.data, 1)[1]
        #print(f"prediction size are {predicted.shape}")
        y_pred = np.append(y_pred, predicted.cpu().numpy())

#print(filenames)

make_submission_file(filenames, y_pred)